In [ ]:
import pandas as pd
from bs4 import BeautifulSoup as bs
import requests
from pandas import json_normalize
import os
import json



> Scraping BusinessInsider. Article principal + 4 articles secondaires.

Titre, contenu, tags



In [ ]:
main="https://www.businessinsider.com/"
response = requests.get(main)

In [ ]:
articles = []
html_main = response.content
soup = bs(html_main, "lxml")
art1 = soup.h1.a["href"]
if "https" not in art1:
  art1 = main+art1[1:]
articles.append(art1)
artsec = soup.ol.find_all("li")

for i in range(4):
  articles.append(artsec[i].a["href"])
articles

['https://markets.businessinsider.com/news/currencies/russian-oil-trade-us-dollars-uae-local-currency-dirhams-roseneft-2022-7',
 'https://www.businessinsider.com/heat-waves-are-dominating-summer-and-its-not-abnormal-anymore-2022-7',
 'https://www.businessinsider.com/heres-how-the-stock-market-could-tumble-or-rally-this-summer-2022-7?r=US&IR=T',
 'https://www.businessinsider.com/fauci-will-step-down-end-of-bidens-first-term-2022-7',
 'https://www.businessinsider.com/travel-prices-flights-hotels-car-rental-cheaper-inflation-weakening-2022-7']

In [ ]:
df_articles = pd.DataFrame(data=None)
df_articles["articles"] = articles
df_articles["title"] = ""
df_articles["body"] = ""
df_articles["tags"] = ""
i = 0
for url in df_articles["articles"]:
  response = requests.get(url)
  html = response.content
  soup = bs(html, "lxml")
  title = soup.h1.text
  df_articles["title"][i] = title

  parags = soup.find_all("p")
  content = ""

  x = "".join((i.text for i in parags))
  df_articles["body"][i] = x

  sectags = soup.find_all("a", {"class" : "category-link not-content-link "})
  first = soup.find_all("a", {"class" : "category-link not-content-link first"})[0].text
  tags = [first]
  print(first)

  for sectag in sectags:
    tags.append(sectag.text)
  df_articles["tags"][i] = tags

  i += 1
df_articles
df_articles.to_csv("articles.csv")

MI Exclusive
Heat wave
Newsletter
Joe Biden
Economy




> API Twitter, recherches basées sur les résultats de scraping

author_id 	created_at 	id 	lang 	text 	score 	link 	keyword



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd drive/MyDrive/Année4
%ls
#df1 = pd.read_excel("tweetbase.xlsx")
#df1

/content/drive/MyDrive/Année4
 bert-nli.pt   Cloud/     InferTest.ipynb   Scala/            Spark/
 Bi/           ez.ipynb  'ML Late'/         Sentiment.ipynb   tut6-model.pt


In [ ]:
for i in range(len(df_articles)):
  #print(len(df_articles['body'][i].split("")))
  df_articles['body'][i] = df_articles['body'][i][0:1800]

In [ ]:
bearer_token = os.environ.get("AAAAAAAAAAAAAAAAAAAAAKjWbgEAAAAAd71Ej2t93WqhATnBrQcgYPsplS8%3DFoRfmQylzxgUS2mOUL6yt6HAsI1JsBmcxMocnVI1w3EBDn0koZ")
#Code sale, token pas caché
bearer_token = "AAAAAAAAAAAAAAAAAAAAAKjWbgEAAAAAd71Ej2t93WqhATnBrQcgYPsplS8%3DFoRfmQylzxgUS2mOUL6yt6HAsI1JsBmcxMocnVI1w3EBDn0koZ"

search_url = "https://api.twitter.com/2/tweets/search/recent"

def bearer_oauth(r):
    """
    Method required by bearer token authentication.
    """

    r.headers["Authorization"] = f"Bearer {bearer_token}"
    r.headers["User-Agent"] = "v2RecentSearchPython"
    return r

def connect_to_endpoint(url, params):
    response = requests.get(url, auth=bearer_oauth, params=params)
    print(response.status_code)
    if response.status_code != 200:
        print(response.status_code)
        raise Exception(response.status_code, response.text)
    return response.json()

df = pd.DataFrame() #IfNoneExists
for i in range(len(df_articles)):
  queries = df_articles['tags'][i]

  score = 5
  link = df_articles['articles'][i]
  # Optional params: start_time,end_time,since_id,until_id,max_results,next_token,
  # expansions,tweet.fields,media.fields,poll.fields,place.fields,user.fields
  for el in queries:

    tweetfields = ['author_id,lang,created_at']
    el = el.replace(" ","")
    query_params = {'query': "#"+el,'tweet.fields': tweetfields, "max_results":10}





    #def main():
    json_response = connect_to_endpoint(search_url, query_params)
    results = json.dumps(json_response, indent=4, sort_keys=True)
    print(results)

    results1 = json.loads(results)
    try:
      df1 = pd.DataFrame.from_dict(results1["data"])
      df1["score"] = score
      df1["link"]= link
      df1["keyword"] = el
      df1["article_text"] = df_articles['body'][i]
      df1["article_title"] = df_articles['title'][i]
      score-=1
      df = df.append(df1)
    except:
      pass


200
{
    "meta": {
        "result_count": 0
    }
}
200
{
    "data": [
        {
            "author_id": "1193148000450813952",
            "created_at": "2022-07-19T15:22:09.000Z",
            "id": "1549414294965583872",
            "lang": "en",
            "text": "RT @DavidSSKCapital: #Ukraine's central bank has sold $12.4 billion of gold reserves since the beginning of Russia's invasion: 'The gold wa\u2026"
        },
        {
            "author_id": "120847369",
            "created_at": "2022-07-19T15:21:15.000Z",
            "id": "1549414069471363072",
            "lang": "en",
            "text": "RT @_huuii: You want more money?  do this...\n\nhttps://t.co/wGq6MlAjhL\n\n#money #value #huuii #quotes #brianfanale #huuiien #markets https://\u2026"
        },
        {
            "author_id": "1191527920193429506",
            "created_at": "2022-07-19T15:20:49.000Z",
            "id": "1549413960566308864",
            "lang": "en",
            "text": "I'm currently fe

In [ ]:
df = df[df["lang"] == 'en'].reset_index(drop = True)
df

,author_id,created_at,id,lang,text,score,link,keyword,article_text,article_title
0,1193148000450813952,2022-07-19T15:22:09.000Z,1549414294965583872,en,RT @DavidSSKCapital: #Ukraine's central bank h...,5,https://markets.businessinsider.com/news/curre...,Markets,Russia is looking to complete an oil deal with...,Russia distances itself from the US dollar fur...
1,120847369,2022-07-19T15:21:15.000Z,1549414069471363072,en,RT @_huuii: You want more money? do this...\n...,5,https://markets.businessinsider.com/news/curre...,Markets,Russia is looking to complete an oil deal with...,Russia distances itself from the US dollar fur...
2,1191527920193429506,2022-07-19T15:20:49.000Z,1549413960566308864,en,I'm currently feeling the (future) pain of all...,5,https://markets.businessinsider.com/news/curre...,Markets,Russia is looking to complete an oil deal with...,Russia distances itself from the US dollar fur...
3,23059499,2022-07-19T15:20:20.000Z,1549413839313088512,en,$CSX $UNP - CSX And Union Pacific: 2 Railroads...,5,https://markets.businessinsider.com/news/curre...,Markets,Russia is looking to complete an oil deal with...,Russia distances itself from the US dollar fur...
4,288638912,2022-07-19T15:20:07.000Z,1549413785416241152,en,#Markets #assetsundermanagement #CoinShares #C...,5,https://markets.businessinsider.com/news/curre...,Markets,Russia is looking to complete an oil deal with...,Russia distances itself from the US dollar fur...
...,...,...,...,...,...,...,...,...,...,...
103,1471340631439990785,2022-07-19T15:21:14.000Z,1549414066082439169,en,RT @ReverieCrimePod: There’s a new episode of ...,4,https://www.businessinsider.com/travel-prices-...,Travel,Inflation is running white-hot. Airports are s...,3 ways summer travel is getting cheaper despit...
104,1448351959669526532,2022-07-19T15:22:41.000Z,1549414429980327936,en,@DanPriceSeattle @AngelaBelcamino What if givi...,3,https://www.businessinsider.com/travel-prices-...,Inflation,Inflation is running white-hot. Airports are s...,3 ways summer travel is getting cheaper despit...
105,979690486872051712,2022-07-19T15:22:33.000Z,1549414397315006470,en,MORE BOE'S BAILEY: 'I WOULD URGE CAUTION IN IN...,3,https://www.businessinsider.com/travel-prices-...,Inflation,Inflation is running white-hot. Airports are s...,3 ways summer travel is getting cheaper despit...
106,109290401,2022-07-19T15:22:21.000Z,1549414346400403456,en,@POTUS 75% of middle class families say their ...,3,https://www.businessinsider.com/travel-prices-...,Inflation,Inflation is running white-hot. Airports are s...,3 ways summer travel is getting cheaper despit...




> Prédiction inférence Tweet <-> corps de l'article



In [ ]:
!pip install transformers
#!pip install torchtext==0.9.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.4 MB 4.2 MB/s 
     |████████████████████████████████| 101 kB 13.8 MB/s 
     |████████████████████████████████| 596 kB 50.7 MB/s 
     |████████████████████████████████| 6.6 MB 44.0 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
import torch.nn as nn
import torch
from transformers import BertModel


class BERTNLIModel(nn.Module):
    def __init__(self,
                 bert_model,
                 hidden_dim,
                 output_dim,
                ):
        
        super().__init__()
        
        self.bert = bert_model
        
        embedding_dim = bert_model.config.to_dict()['hidden_size']
        
        #self.fc = nn.Linear(embedding_dim, hidden_dim)

        #self.fc2 = nn.Linear(hidden_dim, hidden_dim)

        self.out = nn.Linear(embedding_dim, output_dim)
        
        
    def forward(self, sequence, attn_mask, token_type):
        
        #sequence = [sequence len, batch_size]
        #attention_mask = [seq_len, batch_size]
        #token_type = [seq_len, batch_size]
                
        embedded = self.bert(input_ids = sequence, attention_mask = attn_mask, token_type_ids= token_type)[1]
        #print('emb ', embedded.size())

        #self.bert() gives tuple which contains hidden outut corresponding to each token.
        #self.bert()[0] = [seq_len, batch_size, emd_dim]
                
        #embedded = [batch size, emb dim]
        
        #ff = self.fc(embedded)
        #ff = [batch size, hid dim]

        #ff1 = self.fc2(ff)
                
        
        
        output = self.out(embedded)
        #print('output: ', output.size())
        #output = [batch size, out dim]
        
        return output


HIDDEN_DIM = 1024
OUTPUT_DIM = 3

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

bert_model = BertModel.from_pretrained('bert-base-uncased')

model = BERTNLIModel(bert_model,
                         HIDDEN_DIM,
                         OUTPUT_DIM,
                        ).to(device)

model.load_state_dict(torch.load('bert-nli.pt', map_location=torch.device('cpu')))
model.eval()

def predict_inference(premise, hypothesis, model, device):
    
    model.eval()
    
    premise = '[CLS] ' + premise + ' [SEP]'
    hypothesis = hypothesis + ' [SEP]'
    
    prem_t = tokenize_bert(premise)
    hypo_t = tokenize_bert(hypothesis)
    
    #print(len(prem_t), len(hypo_t))
    
    prem_type = get_sent1_token_type(prem_t)
    hypo_type = get_sent2_token_type(hypo_t)
    
    #print(len(prem_type), len(hypo_type))
    
    indexes = prem_t + hypo_t
    
    indexes = tokenizer.convert_tokens_to_ids(indexes)
    #print(indexes)
    indexes_type = prem_type + hypo_type
    #print(indexes_type)
    
    attn_mask = get_sent2_token_type(indexes)
    #print(attn_mask)
    
    #print(len(indexes))
    #print(len(indexes_type))
    #print(len(attn_mask))

    #seq = '[CLS] '+ premise + ' [SEP] '+ hypothesis 

    #tokens = tokenizer.tokenize(seq)

    #indexes = tokenizer.convert_tokens_to_ids(tokens)
    
    indexes = torch.LongTensor(indexes).unsqueeze(0).to(device)
    indexes_type = torch.LongTensor(indexes_type).unsqueeze(0).to(device)
    attn_mask = torch.LongTensor(attn_mask).unsqueeze(0).to(device)
    
    #print(indexes.size())
    
    prediction = model(indexes, attn_mask, indexes_type)
    
    prediction = prediction.argmax(dim=-1).item()
    
    return LABEL[prediction]

def tokenize_bert(sentence):
    tokens = tokenizer.tokenize(sentence)
    #tokens = tokenizer.encode(tokens, padding=True, truncation=True,max_length=400, add_special_tokens = True)
    return tokens

def get_sent1_token_type(sent):
    try:
        return [0]* len(sent)
    except:
        return []

def get_sent2_token_type(sent):
    try:
        return [1]* len(sent)
    except:
        return []


from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')




LABEL = ['entailment', 'contradiction', 'neutral']
LABEL

premise = 'a man sitting on a green bench.'
hypothesis = 'a woman sitting on a green bench.'

predict_inference(premise, hypothesis, model, device)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

'contradiction'

In [ ]:
df["prediction"] = ""

for i in range(len(df)):
  premise = df["article_text"][i]
  hypothesis = df["text"][i]
  predicted = predict_inference(premise, hypothesis, model, device)
  df['prediction'][i] = predicted

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [ ]:
df

,author_id,created_at,id,lang,text,score,link,keyword,article_text,article_title,prediction
0,1193148000450813952,2022-07-19T15:22:09.000Z,1549414294965583872,en,RT @DavidSSKCapital: #Ukraine's central bank h...,5,https://markets.businessinsider.com/news/curre...,Markets,Russia is looking to complete an oil deal with...,Russia distances itself from the US dollar fur...,contradiction
1,120847369,2022-07-19T15:21:15.000Z,1549414069471363072,en,RT @_huuii: You want more money? do this...\n...,5,https://markets.businessinsider.com/news/curre...,Markets,Russia is looking to complete an oil deal with...,Russia distances itself from the US dollar fur...,neutral
2,1191527920193429506,2022-07-19T15:20:49.000Z,1549413960566308864,en,I'm currently feeling the (future) pain of all...,5,https://markets.businessinsider.com/news/curre...,Markets,Russia is looking to complete an oil deal with...,Russia distances itself from the US dollar fur...,contradiction
3,23059499,2022-07-19T15:20:20.000Z,1549413839313088512,en,$CSX $UNP - CSX And Union Pacific: 2 Railroads...,5,https://markets.businessinsider.com/news/curre...,Markets,Russia is looking to complete an oil deal with...,Russia distances itself from the US dollar fur...,neutral
4,288638912,2022-07-19T15:20:07.000Z,1549413785416241152,en,#Markets #assetsundermanagement #CoinShares #C...,5,https://markets.businessinsider.com/news/curre...,Markets,Russia is looking to complete an oil deal with...,Russia distances itself from the US dollar fur...,contradiction
...,...,...,...,...,...,...,...,...,...,...,...
103,1471340631439990785,2022-07-19T15:21:14.000Z,1549414066082439169,en,RT @ReverieCrimePod: There’s a new episode of ...,4,https://www.businessinsider.com/travel-prices-...,Travel,Inflation is running white-hot. Airports are s...,3 ways summer travel is getting cheaper despit...,contradiction
104,1448351959669526532,2022-07-19T15:22:41.000Z,1549414429980327936,en,@DanPriceSeattle @AngelaBelcamino What if givi...,3,https://www.businessinsider.com/travel-prices-...,Inflation,Inflation is running white-hot. Airports are s...,3 ways summer travel is getting cheaper despit...,contradiction
105,979690486872051712,2022-07-19T15:22:33.000Z,1549414397315006470,en,MORE BOE'S BAILEY: 'I WOULD URGE CAUTION IN IN...,3,https://www.businessinsider.com/travel-prices-...,Inflation,Inflation is running white-hot. Airports are s...,3 ways summer travel is getting cheaper despit...,neutral
106,109290401,2022-07-19T15:22:21.000Z,1549414346400403456,en,@POTUS 75% of middle class families say their ...,3,https://www.businessinsider.com/travel-prices-...,Inflation,Inflation is running white-hot. Airports are s...,3 ways summer travel is getting cheaper despit...,contradiction


In [ ]:
df["title_prediction"] = ""

for i in range(len(df)):
  premise = df["article_title"][i]
  hypothesis = df["text"][i]
  predicted = predict_inference(premise, hypothesis, model, device)
  df['title_prediction'][i] = predicted

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [ ]:
df[df["prediction"]== "entailment"]

,author_id,created_at,id,lang,text,score,link,keyword,article_text,article_title,prediction,title_prediction
8,23059499,2022-07-19T15:17:56.000Z,1549413236444766208,en,$PNC - PNC Financial Benefiting From A Hot Mai...,5,https://markets.businessinsider.com/news/curre...,Markets,Russia is looking to complete an oil deal with...,Russia distances itself from the US dollar fur...,entailment,neutral


In [ ]:


premise = 'I am lying down on bed.'
hypothesis = 'I am resting on bed.'

predict_inference(premise, hypothesis, model, device)



'entailment'

In [ ]:
dftemp = df

dftemp["prediction"] = dftemp["prediction"].replace({"neutral" : 1, "contradiction" : 0, "entailment" : 3})

dftemp["title_prediction"] = dftemp["title_prediction"].replace({"neutral" : 1, "contradiction" : 0, "entailment" : 3})

predicted_article = dftemp.groupby("article_title").mean()
predicted_article

,score,prediction,title_prediction
article_title,,,
3 ways summer travel is getting cheaper despite airline chaos and sky-high inflation,4.263158,0.315789,0.105263
Fauci says he will step down by the end of Biden's first term,4.076923,0.461538,0.307692
"Heat waves are dominating summer, killing thousands, and fueling wildfires. The world needs to prepare for more.",3.892857,0.571429,0.500000
Here's what's in store for the stock market for the rest of this summer and beyond,4.500000,0.500000,0.375000
Russia distances itself from the US dollar further as it moves to trade oil with India using the UAE's local currency: report,4.074074,0.740741,0.555556


In [ ]:
dftemp

,author_id,created_at,id,lang,text,score,link,keyword,article_text,article_title,prediction,title_prediction
0,1193148000450813952,2022-07-19T15:22:09.000Z,1549414294965583872,en,RT @DavidSSKCapital: #Ukraine's central bank h...,5,https://markets.businessinsider.com/news/curre...,Markets,Russia is looking to complete an oil deal with...,Russia distances itself from the US dollar fur...,0,0
1,120847369,2022-07-19T15:21:15.000Z,1549414069471363072,en,RT @_huuii: You want more money? do this...\n...,5,https://markets.businessinsider.com/news/curre...,Markets,Russia is looking to complete an oil deal with...,Russia distances itself from the US dollar fur...,1,1
2,1191527920193429506,2022-07-19T15:20:49.000Z,1549413960566308864,en,I'm currently feeling the (future) pain of all...,5,https://markets.businessinsider.com/news/curre...,Markets,Russia is looking to complete an oil deal with...,Russia distances itself from the US dollar fur...,0,0
3,23059499,2022-07-19T15:20:20.000Z,1549413839313088512,en,$CSX $UNP - CSX And Union Pacific: 2 Railroads...,5,https://markets.businessinsider.com/news/curre...,Markets,Russia is looking to complete an oil deal with...,Russia distances itself from the US dollar fur...,1,0
4,288638912,2022-07-19T15:20:07.000Z,1549413785416241152,en,#Markets #assetsundermanagement #CoinShares #C...,5,https://markets.businessinsider.com/news/curre...,Markets,Russia is looking to complete an oil deal with...,Russia distances itself from the US dollar fur...,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
103,1471340631439990785,2022-07-19T15:21:14.000Z,1549414066082439169,en,RT @ReverieCrimePod: There’s a new episode of ...,4,https://www.businessinsider.com/travel-prices-...,Travel,Inflation is running white-hot. Airports are s...,3 ways summer travel is getting cheaper despit...,0,0
104,1448351959669526532,2022-07-19T15:22:41.000Z,1549414429980327936,en,@DanPriceSeattle @AngelaBelcamino What if givi...,3,https://www.businessinsider.com/travel-prices-...,Inflation,Inflation is running white-hot. Airports are s...,3 ways summer travel is getting cheaper despit...,0,0
105,979690486872051712,2022-07-19T15:22:33.000Z,1549414397315006470,en,MORE BOE'S BAILEY: 'I WOULD URGE CAUTION IN IN...,3,https://www.businessinsider.com/travel-prices-...,Inflation,Inflation is running white-hot. Airports are s...,3 ways summer travel is getting cheaper despit...,1,0
106,109290401,2022-07-19T15:22:21.000Z,1549414346400403456,en,@POTUS 75% of middle class families say their ...,3,https://www.businessinsider.com/travel-prices-...,Inflation,Inflation is running white-hot. Airports are s...,3 ways summer travel is getting cheaper despit...,0,0
